In [1]:
import pandas as pd
from ast import literal_eval
import csv
import pymssql
from datetime import datetime

In [2]:
df = pd.read_csv('../data/movies_new.csv')

df = df[['id', 'title', 'release year']]

movies = []

for i in range(len(df['release year'])):
    df.at[i, 'release year'] = ''.join(filter(str.isdigit, str(df.at[i, 'release year'])))

for index, row in df.iterrows():
    year_date = datetime.strptime(row['release year'], '%Y')
    movie = {"id": row['id'], 'title': row['title'], 'release_year': year_date}
    movies.append(movie)         

In [3]:
# Connect to MSSQL database
server = 'localhost'
database = 'BockBluster'
username = 'sa'
password = 'thisIsSuperStrong1234'

connection = pymssql.connect(server=server, database=database, user=username, password=password)
cursor = connection.cursor()

# Insert dictionaries into the table
cursor = connection.cursor()
for dictionary in movies:
    cursor.execute("INSERT INTO movie (movie_id, title, release_date) VALUES (%s, %s, %s)", (dictionary['id'], dictionary['title'], dictionary['release_year']))

# Commit the changes and close the connection
connection.commit()
connection.close()